## Presentation Type Classifier (with Hidden Types)

#### Also Network Visualization with Louvain Communities--See below

* May 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by **more than 10 bars (80 offsets)**, the tool resets to a new pattern
    * Finds **time intervals between entries* (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds **melodic intervals between first note of successive entries** in each pattern (like `P-5, P-8`)
    * Counts **number of entries**
    * Provides **offset and measure/beat locations**
    * Sorts all presentation types by the **order in which they appear in the piece**
    * Reports **voice names of the entries, in order of their appearance**
    * **Omits singleton soggetti** (just one entry of a given motive in isolation)
    
* Options
    
    * set the **length of the soggetti** with `melodic_ngram_length` (4 is a good default)
    * set the **maximum difference between similar soggetti** with `edit_distance_threshold` (0 will yield only identical soggetti; 1 will allow one total integer difference across all the intervals)
    * for **chromatic vs diatonic, compound, and zero-based diatonic in soggetti**, use interval_settings (`kind= 'd', or 'c', or 'z'`)
    * to include all the **hidden PENs and IDS** (those found within longer Fugas, use `include_hidden_types == True`. That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above. For faster (and simpler) listing of points of imitation **without hidden forms**, use `include_hidden_types == False`
    * **combineUnisons** via the initial import of the NotesRests.  Select this with `True` or `False`.  Combining Unisons can be a good way to identify connections across compositions
    * **Limit_to_Entries** allows you to choose whether the P Type finder considers **only soggetti that enter after a rest or section break** or instead considers **all ngrams** as a moving window in each voice.  The latter will likely produce large numbers of overlapping Presentation Types of the same basic type and time/melodic entries.  

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [2]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0019.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0019.mei
{'title': 'Veni sponsa Christi', 'composer': 'Palestrina, Giovanni Pierluigi da'}


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



In [25]:
include_hidden_types = False
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 2
limit_to_entries = True
nr = piece.getNoteRest(combineUnisons=combine_unisons)
dur = piece.getDuration(df=nr)
mel = piece.getMelodic(df=nr, kind='d', end=False)

if limit_to_entries == True:
    entries = piece.getEntries(mel_ng)
if limit_to_entries == False:
    entries = piece.getNgrams(df=mel, n=melodic_ngram_length)

# entries
output = classify_entries_as_presentation_types(piece, nr, mel_ng, entries, edit_distance_threshold, include_hidden_types)



In [26]:
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[(-3, 3, 2, -2), (-3, 2, 2, -2), (-2, 2, 2, -2...","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2), (-2, 2, 2, -2), (-2, 2, 3, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8,True
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Bassus, Altus, Cantus, Bassus]",FUGA,6,False
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,386.0,"[49/2.0, 50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4....","[M-2, P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[386.0, 394.0, 406.0, 418.0, 434.0, 446.0, 464...","[(-3, 3, 2, -2), (-3, 2, 2, -2), (-2, 2, 2, -2...","[8.0, 12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0,...","[Tenor, Altus, Cantus, Bassus, Tenor, Altus, B...",FUGA,10,True


### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [24]:


piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']

In [26]:

git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
mei = '.mei'
include_hidden_types = True
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 0
final = pd.DataFrame()

for piece in piece_list:
    file_url = git_prefix + piece 
    url = git_prefix + piece + mei
    piece = importScore(url)   
    nr = piece.getNoteRest(combineUnisons=combine_unisons)
    dur = piece.getDuration(df=nr)
    mel = piece.getMelodic(df=nr, kind='d', end=False)
    mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
    if limit_to_entries == True:
        entries = piece.getEntries(mel_ng)
    if limit_to_entries == False:
        entries = piece.getNgrams(df=mel, n=melodic_ngram_length)

    output = classify_entries_as_presentation_types(piece, nr, mel_ng, entries, edit_distance_threshold, include_hidden_types)


    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)
# final.to_csv("corpus_classified.csv")
final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


# Network Visualization with Louvain Communities

* This will only make sense for SETS of pieces!

### Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

In [54]:
output["MINT"] = output["Melodic_Entry_Intervals"].apply(joiner)
output["TINT"] = output["Time_Entry_Intervals"].apply(joiner)

output['SOG'] = output['Soggetti'].apply(clean_melody_new)
output['ALL'] = output["MINT"] + '_' + output["TINT"] + '_' + output['SOG']

output["ALL_INT"] = output["MINT"] + '_' + output["TINT"]
output["MINT_SOG"] = output["MINT"] + '_' + output["SOG"]
output["TINT_SOG"] = output["TINT"] + '_' + output["SOG"]
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG,MINT_SOG,TINT_SOG
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8, P1, P8, P-5, P-4,...","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0, 150...","[(-3, 3, 2, -2), (-3, 2, 2, -2), (-2, 2, 2, -2...","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0, 22.0, 16.0,...","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,15,True,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...,-3_3_2_-2,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,P-5_P-4_P-5_P8_P5_P-8_P1_P8_P-5_P-4_P-5_P8_P8_...,8.0_32.0_8.0_36.0_20.0_24.0_22.0_16.0_8.0_24.0...
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2), (-2, 2, 2, -2), (-2, 2, 3, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8,True,P8_P-5_P-4_P-5_P8_P8_P-11,16.0_8.0_24.0_16.0_16.0_20.0_24.0,-3_2_2_-2,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_16.0_8.0_24.0_16.0_1...,P8_P-5_P-4_P-5_P8_P8_P-11_-3_2_2_-2,P8_P-5_P-4_P-5_P8_P8_P-11_-3_2_2_-2,16.0_8.0_24.0_16.0_16.0_20.0_24.0_-3_2_2_-2
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[(-3, 3, -2, -2)]","[4.0, 28.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Bassus, Altus, Cantus, Bassus]",FUGA,6,False,P-5_P-5_P8_P8_P-11,4.0_28.0_14.0_18.0_20.0,-3_3_-2_-2,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0_-3_...,P-5_P-5_P8_P8_P-11_4.0_28.0_14.0_18.0_20.0,P-5_P-5_P8_P8_P-11_-3_3_-2_-2,P-5_P-5_P8_P8_P-11_-3_3_-2_-2,4.0_28.0_14.0_18.0_20.0_-3_3_-2_-2
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,386.0,"[49/2.0, 50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4....","[M-2, P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[386.0, 394.0, 406.0, 418.0, 434.0, 446.0, 464...","[(-3, 3, 2, -2), (-3, 2, 2, -2), (-2, 2, 2, -2...","[8.0, 12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0,...","[Tenor, Altus, Cantus, Bassus, Tenor, Altus, B...",FUGA,10,True,M-2_P5_P-8_P4_P5_P-8_P8_P-4_P-5,8.0_12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0,-3_3_2_-2,M-2_P5_P-8_P4_P5_P-8_P8_P-4_P-5_8.0_12.0_12.0_...,M-2_P5_P-8_P4_P5_P-8_P8_P-4_P-5_8.0_12.0_12.0_...,M-2_P5_P-8_P4_P5_P-8_P8_P-4_P-5_-3_3_2_-2,M-2_P5_P-8_P4_P5_P-8_P8_P-4_P-5_-3_3_2_-2,8.0_12.0_12.0_16.0_12.0_18.0_22.0_4.0_16.0_-3_...


### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [64]:

output_grouped = output.groupby('SOG')['Title'].apply(list).reset_index()
output_grouped


,SOG,Title
0,-3_2_2_-2,[Veni sponsa Christi]
1,-3_3_-2_-2,[Veni sponsa Christi]
2,-3_3_2_-2,"[Veni sponsa Christi, Veni sponsa Christi]"


In [65]:
pairs = output_grouped.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0                                              []
1                                              []
2    [(Veni sponsa Christi, Veni sponsa Christi)]
Name: Title, dtype: object

In [66]:
pairs2 = pairs.explode().dropna()
pairs2

2    (Veni sponsa Christi, Veni sponsa Christi)
Name: Title, dtype: object

In [67]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([('Veni sponsa Christi', 'Veni sponsa Christi')], dtype=object)

In [63]:
pd.Series(unique_pairs).isna().sum()

0

In [23]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [24]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [25]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [26]:
pyvis_graph.from_nx(G)

In [27]:
pyvis_graph.show('MINT.html')


#### Below is Development Work

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [ ]:
# This test works

points = pd.DataFrame()
points2 = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    # here is the list of starting offsets of the original set of entries:  slist
    slist = split_list[0]
    temp = temp_dict_of_details(slist, entry_array, det, matches)

    points = points.append(temp, ignore_index=True)
    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
    points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
    points = points[points['Offsets'].apply(len) > 1]

    test = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]

    l = len(test)  
    for item in range(3, l):
        list_combinations = list(combinations(test, item))
        for group in list_combinations:
            group_time_ints = numpy.diff(group).tolist()
            hidden_type = classify_by_offset(group_time_ints)
            for item in group:
    #         print(item)
                array = group[entry_array.index.get_level_values(0).isin(item)]
                short_offset_list = array.index.to_list()
                time_ints = numpy.diff(array.index).tolist()
                voice_list = array['voice'].to_list()
                if 'PEN' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
                if 'ID' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
# len(split_list[0])           

In [ ]:
#  This shows how the classifier works:

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
# This shows how combinations works for a given set of time intervals
offset_diffs = [12.0, 32.0, 12.0, 4.0]
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)

In [ ]:
slist = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]
l = len(slist)
# for r in range(3, 6):
list_combinations = list(combinations(slist, 4))
#     for tiny_list in list_combinations:

In [ ]:
print(list_combinations)

In [ ]:
list_offsets = [294.0, 298.0, 306.0, 310.0]

In [ ]:
offset_diffs = [4, 5, 6]

In [ ]:
alt_list = offset_diffs[::2]

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')